In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# birdirectional LSTM (Long Short Term Memory)!pip install keras-self-attention
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM, Bidirectional

import  os
from keras.models import Sequential, load_model

from keras.regularizers import L1L2
from sklearn.preprocessing import MinMaxScaler
import  pandas as pd
import numpy as np

from keras_self_attention import SeqSelfAttention

dic={'batch_size': 116.0, 'dropout1': 0.31955278974391277, 'dropout3': 0.6936110582868452, 'dropout4': 0.5044383573441469, 'units1': 428.0, 'units3': 290.0, 'units4': 202.0}
forword=30
look_back=120
tr_size=0.86
ep=50


df=pd.read_excel('/kaggle/input/oilprice1/5.xlsx')
df1=df.copy()


df1=df1[['日期（月度）','USA_output','OPEC_output','demand_current','supply_current']]
#df1=df1.bfill()
df1=df1.dropna()


df1=df1.rename(columns={"日期（月度）":'日期（日期）'})
df1=df1.set_index('日期（日期）')


df1=df1.asfreq(freq='d')

df1=df1.bfill()
#print(df1.head(40))

df=df.set_index('日期（日期）')


df=df[['Brent','USD','CRB','Premium','trade','BDI']]


df=df.ffill()
df=df.bfill()



df=df.join(df1)

df=df.dropna()
df=df.astype('float64')

df_raw=df.copy()
df = df.diff().dropna()

#df['flag']=[ (lambda y: 1 if (y%7==6)|(y%7==0) else 0)(x) for x in range(df.shape[0])]
print(df.shape[0])
print(df_raw.shape[0])




#print(df)



#df=df.iloc[0:50]



dataset=df.values
#dataset = dataset.astype('float32')
#print(dataset)
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)






train_size = int(len(dataset) * tr_size)
trainlist = dataset[:train_size]
testlist = dataset[train_size:]



def create_dataset(dataset, look_back,forword):

        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-forword):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[(i + look_back):(i+look_back+forword)])
        return np.array(dataX),np.array(dataY)


trainX,trainY  = create_dataset(trainlist,look_back,forword)
testX,testY = create_dataset(testlist,look_back,forword)


trainY=np.reshape(trainY,(trainY.shape[0],-1))

#print(trainX)
#print(trainY)


def trainModel():

    model = Sequential()
    '''
    model.add(LSTM(
    20,
    input_shape=(trainX.shape[1], trainX.shape[2])))
    '''
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2,kernel_regularizer=L1L2(l1=0.001, l2=0.001), recurrent_regularizer=L1L2(l1=0.001, l2=0.001), bias_regularizer=L1L2(l1=0.001, l2=0.001),return_sequences=True), input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2,kernel_regularizer=L1L2(l1=0.001, l2=0.001), recurrent_regularizer=L1L2(l1=0.001, l2=0.001), bias_regularizer=L1L2(l1=0.001, l2=0.001),return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid',kernel_regularizer=L1L2(l1=0.0001, l2=0.0001),bias_regularizer=L1L2(l1=0.0001, l2=0.0001),attention_regularizer_weight=1e-4))

    model.add(Flatten())
    model.add(Dense(trainY.shape[1]))
    model.add(Activation("relu"))
    model.compile(loss='mse', optimizer='adam')
    

    return model



model=trainModel()

model.fit(trainX, trainY, epochs=ep, batch_size=int(dic['batch_size']), verbose=2)



model.save('model2.h5')    



#y_hat  =  model.predict(np.reshape(trainX[0:2],(-1,trainX.shape[1],trainX.shape[2])))
y_hat  =  model.predict(testX)



print(y_hat)
def retrive(hat):
    hat=np.reshape(hat,(-1,df.shape[1]))
    hat=scaler.inverse_transform(hat)
    hat=np.reshape(hat,(-1,forword,df.shape[1]))
    ind=train_size+look_back-1
    result=[]
    for y in range(hat.shape[0]):
        ind_temp=ind+y
        temp=np.array([ hat[y][x][0] for x in range(forword)])
        las=df_raw.iloc[(ind_temp+1)]['Brent']
        temp=np.insert(temp,0,las)
        temp=np.cumsum(temp)
        result.append(temp[1:])
    #print(np.array(result))
    return np.array(result)


#print(df_raw.iloc[train_size:])

diff=retrive(y_hat)-retrive(testY)


mae=(np.abs(diff)).mean()
#rmse=np.sqrt((diff**2).mean())
print(mae)
fo=open('1.txt','w')
fo.write(repr(mae))
fo.close()

  Created wheel for keras-self-attention: filename=keras_self_attention-0.41.0-cp36-none-any.whl size=17289 sha256=84871831c188abb68de2746af4b296f396c1724261c422c14f88f65f332594e4
  Stored in directory: /tmp/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-self-attention


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

6574
6575
Epoch 1/50
 - 196s - loss: 33.7477
Epoch 2/50
 - 187s - loss: 6.9953
Epoch 3/50
 - 185s - loss: 3.1631
Epoch 4/50
 - 186s - loss: 2.2570
Epoch 5/50
 - 186s - loss: 1.9231
Epoch 6/50
 - 185s - loss: 1.7425
Epoch 7/50
 - 186s - loss: 1.6058
Epoch 8/50
 - 184s - loss: 1.4849
Epoch 9/50
 - 186s - loss: 1.3740
Epoch 10/50
 - 186s - loss: 1.2718
Epoch 11/50
 - 185s - loss: 1.1765
Epoch 12/50
 - 185s - loss: 1.0881
Epoch 13/50
 - 185s - loss: 1.0058
Epoch 14/50
 - 185s - loss: 0.9288
Epoch 15/50
 - 185s - loss: 0.8569
Epoch 16/50
 - 183s - loss: 0.7892
Epoch 17/50
 - 186s - loss: 0.7257
Epoch 18/50
 - 184s - loss: 0.6662
Epoch 19/50
 - 186s - loss: 0.6106
Epoch 20/50
 - 186s - loss: 0.5589
Epoch 21/50
 - 184s - loss: 0.5107
Epoch 22/50
 - 183s - loss: 0.4660
Epoch 23/50
 - 184s - loss: 0.4244
Epoch 24/50
 - 182s - loss: 0.3859
Epoch 25/50
 - 182s - loss: 0.3505
Epoch 26/50
 - 182s - loss: 0.3175
Epoch 27/50
 - 186s - loss: 0.2881
Epoch 28/50
 - 185s - loss: 0.2720
Epoch 29/50
 - 185